In [1]:
import pandas as pd

In [54]:
# Function for parsing datetimes from original csv file
date_parse = lambda x : pd.to_datetime(x, format="%a, %Y-%m-%d %H:%M:%S")

cons = pd.read_csv('cons.csv', parse_dates=['create_dt', 'modified_dt'], date_parser=date_parse)
cons_email = pd.read_csv('cons_email.csv')
cons_email_chapter = pd.read_csv('cons_email_chapter_subscription.csv')

In [57]:
# Filter for primary emails
cons_email_partial = cons_email[cons_email['is_primary'] == 1]

# Filter for columns needed
cons_email_partial = cons_email_partial[['cons_email_id', 'cons_id', 'email']]

In [58]:
# Filter for desired `chapter_id`
cons_email_chapter_partial = cons_email_chapter[cons_email_chapter['chapter_id'] == 1]

# Filter for columns needed
cons_email_chapter_partial = cons_email_chapter_partial[['cons_email_id', 'isunsub']]

In [59]:
email_df = cons_email_partial.merge(cons_email_chapter_partial, on="cons_email_id", how="left")

In [60]:
# Any email not in `cons_email_chapter_subscription.csv` is assumed to still be subscribed where chapter_id is 1.
email_df['isunsub'] = email_df['isunsub'].fillna(0)

# Filter for columns needed
email_df = email_df[['cons_id', 'email', 'isunsub']]

In [61]:
final_df = cons.merge(email_df, on="cons_id", how="left")

# Filter for columns needed
final_df = final_df[['email', 'source','isunsub', 'create_dt', 'modified_dt']]

# Save as csv file in working directory
final_df.to_csv('people.csv', index=False)

In [62]:
# Aggregate by date of creation
acq_df = final_df.groupby([final_df['create_dt'].dt.date]).count()[['create_dt']]

# Format dataframe
acq_df.index.rename('acquisition_date', inplace=True)
acq_df.columns = ['acquisitions']

# Save as csv file in working directory
acq_df.to_csv('acquisition_facts.csv')